![Neptune + Prophet](https://neptune.ai/wp-content/uploads/2023/09/prophet-1.svg)

# Neptune + Prophet

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/prophet/notebooks/Neptune_prophet.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/prophet/notebooks/Neptune_prophet.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/fbprophet-integration/runs/details?viewId=standard-view&detailsTab=metadata&shortId=FBPROP-3211"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/prophet/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to log the following Prophet metadata with Neptune, using a single function:

* Regression summary metadata
* Input and forecast Dataframes
* Forecast, components and residual diagnostics plots
* Model configuration
* Serialized model

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U -q "neptune[prophet]"

In [ ]:
import neptune

In [ ]:
# Run this if you get a `RuntimeError: main thread is not in main loop` error
import matplotlib

matplotlib.use("Agg")

## Initialize Neptune and create new run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["prophet", "notebook"],  # optional
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
run = neptune.init_run(
    project="common/fbprophet-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["prophet", "notebook"],  # optional
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_R.csv"
)

# Market capacity
df["cap"] = 8.5

In [ ]:
from prophet import Prophet

In [ ]:
model = Prophet(growth="logistic")
model.fit(df)

In [ ]:
forecast = model.predict(df)

### Log summary metadata

Log the Prophet regression summary to Neptune using the `create_summary()` function:


In [ ]:
import neptune.integrations.prophet as npt_utils

run["prophet_summary"] = npt_utils.create_summary(
    model=model,
    df=df,
    fcst=forecast,
    log_interactive=False,
)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

### Explore results in Neptune

You just learned how to log the Prophet regressor summary to Neptune using a single function. 

Click on the link that was printed to the console or [go here](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdiagnostics_charts%2F) to explore a run similar to yours. In particular check:

* [logged dataframes](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdataframes%2F)
* [logged regressor visualizations](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdiagnostics_charts%2F)
* [logged model config](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fmodel%2Fmodel_config%2F)
* [logged serialized model](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fmodel%2F&attribute=serialized_model)
* [logged metadata](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=sys)
* [logged code and git metadata](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/source-code?path=source_code&file=Neptune_prophet.py)

## More Options

The Neptune + Prophet integration also lets you log only specific metadata of your choice, by using additional methods.
Below are a few examples.

For the full example, see the [Neptune docs](https://docs.neptune.ai/integrations-and-supported-tools/model-training/prophet).

### Before you start: create, fit and predict a Prophet model

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_R.csv"
)

# Market capacity
df["cap"] = 8.5

model = Prophet(growth="logistic")
model.fit(df)
forecast = model.predict(df)

In [ ]:
import neptune.integrations.prophet as npt_utils

run = neptune.init_run(
    project="common/fbprophet-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["prophet", "notebook"],  # optional
)

Open the link above to see the metadata logging results, as we add them below.

### Log Prophet plots

In [ ]:
run["forecast_plots"] = npt_utils.create_forecast_plots(model, forecast, log_interactive=False)

In [ ]:
run["forecast_components"] = npt_utils.get_forecast_components(model, forecast)

In [ ]:
run["residual_diagnostics_plot"] = npt_utils.create_residual_diagnostics_plots(
    forecast, df.y, log_interactive=False
)

### Log Prophet model configuration 

In [ ]:
run["model_config"] = npt_utils.get_model_config(model)

### Log Prophet serialized model

In [ ]:
run["model"] = npt_utils.get_serialized_model(model)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()